Data Analysis Pipeline:


*  Clone the repo to carry the dependencies over and to invoke imagene.py.
*  Import all necessary modules.


*   Simulate data for moderate and ancient selection selrange = seq 0 200 200 / timerange = 0.1, 100kya

*   Build and compile baseline model.Gather Baseline metrics e.g accuracy, F1 and confusion matrixes


*   Train model on training data and gather metrics.

*   Test trained network on unseen data. The 10th Batch of Data













In [1]:
# Cloning the GitHub repository
!git clone https://github.com/Djinho/EvoNet-CNN-Insight.git

# Change directories into the specified directory
%cd EvoNet-CNN-Insight/model_training/Ancient_moderate



Cloning into 'EvoNet-CNN-Insight'...
remote: Enumerating objects: 3732, done.
remote: Counting objects: 100% (548/548), done.
remote: Compressing objects: 100% (301/301), done.
remote: Total 3732 (delta 372), reused 314 (delta 247), pack-reused 3184
Receiving objects: 100% (3732/3732), 4.74 GiB | 26.58 MiB/s, done.
Resolving deltas: 100% (1273/1273), done.
Updating files: 100% (155/155), done.
/content/EvoNet-CNN-Insight/model_training/Ancient_moderate


In [2]:
import os
import gzip
import _pickle as pickle

import numpy as np
import scipy.stats
import arviz

import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, regularizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, models, optimizers



# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

from sklearn.model_selection import train_test_split

import itertools
import matplotlib.pyplot as plt
import skimage.transform
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, roc_curve, auc
import pydot  # Optional, but required by keras to plot the model

In [4]:
%run -i ../../ImaGene.py

In [ ]:
path = './'

ImaGene uses msms for simulations to train networks. Run generate_dataset.sh with params.txt. Simulates 200,000 loci (80kbp) under neutral evolution or positive selection (additive effect, s=200). Selection starts 100kya (allele frequency 0.01), mutation rate 1.5e-8, recombination rate 1e-8. Follows a 3-epoch model (Marth et al. 2004) for a European population, sampling 198 chromosomal copies. The script splits simulations into batches for training.


In [ ]:
import subprocess

# Run the simulation script with specified parameters
subprocess.call("bash ../generate_dataset.sh params_Ancient_moderate.txt".split());


In [ ]:

path_sim = './'

In [ ]:
# Initialize the ImaFile object with specified folder, number of samples, and model name
file_sim = ImaFile(simulations_folder=path_sim + 'AM/Simulations1', nr_samples=198, model_name='Marth-3epoch-CEU');


In [ ]:
# Read simulation data from the specified folder, filtering by 'selection_coeff_hetero' and limiting to 2000 replications
gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=2000);


In [ ]:
# Filter the simulation data to include only those with a frequency of 0.01
gene_sim.filter_freq(0.01);

# Sort the filtered simulation data by 'rows_freq'
gene_sim.sort('rows_freq');



In [ ]:
# Resize the simulation data to a matrix with dimensions 198x192
gene_sim.resize((198, 192));

# Convert the simulation data, applying a flip transformation
gene_sim.convert(flip=True);


shuffle images before using them for training network.


In [ ]:
# Create a random subset of the simulation data
gene_sim.subset(get_index_random(gene_sim));

# Convert the targets of the simulation data to binary
gene_sim.targets = to_binary(gene_sim.targets);

# Save the processed simulation data to a binary file
gene_sim.save(file=path + 'gene_sim.binary');

# Load the simulation data from the saved binary file
gene_sim = load_imagene(file=path + 'gene_sim.binary');


Build and compile the model.

In [ ]:
model = models.Sequential([
                    layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.005, l2=0.005), padding='valid', input_shape=gene_sim.data.shape[1:]),
                    layers.MaxPooling2D(pool_size=(2,2)),
                    layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.005, l2=0.005), padding='valid'),
                    layers.MaxPooling2D(pool_size=(2,2)),
                    layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.005, l2=0.005), padding='valid'),
                    layers.MaxPooling2D(pool_size=(2,2)),
                    layers.Flatten(),
                    layers.Dense(units=128, activation='relu'),
                    layers.Dense(units=1, activation='sigmoid')])
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

summary of the model

In [ ]:
model.summary()


In [ ]:
net_LCT = ImaNet(name='[C32+P]x2+[C64+P]+D128')

train batches from 1-9

In [ ]:
import csv

# Initialize variables to store training metrics and total number of epochs
total_epochs = 0
losses = []
val_losses = []
accuracies = []
val_accuracies = []

# Loop to iterate through different simulation datasets
i = 1
while i < 10:
    print(i)

    # Load simulation data for the current iteration
    file_sim = ImaFile(simulations_folder=path_sim + 'AM/Simulations' + str(i), nr_samples=198, model_name='Marth-3epoch-CEU')
    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=2000)

    # Filter, sort, and preprocess the simulation data
    gene_sim.filter_freq(0.01)
    gene_sim.sort('rows_freq')
    gene_sim.resize((198, 192))
    gene_sim.convert(flip=True)

    # Create a random subset and convert targets to binary
    gene_sim.subset(get_index_random(gene_sim))
    gene_sim.targets = to_binary(gene_sim.targets)

    # Train the model for one epoch on the current dataset
    score = model.fit(gene_sim.data, gene_sim.targets, batch_size=64, epochs=1, verbose=1, validation_split=0.10)

    # Append the loss, validation loss, accuracy, and validation accuracy values for each epoch
    losses.extend(score.history['loss'])
    val_losses.extend(score.history['val_loss'])
    accuracies.extend(score.history['accuracy'])
    val_accuracies.extend(score.history['val_accuracy'])

    # Update the total number of epochs trained
    total_epochs += len(score.history['loss'])

    # Update scores in the net_LCT object (assumed to be defined elsewhere)
    net_LCT.update_scores(score)

    # Increment the loop counter
    i += 1

# Print the total number of epochs
print(f'Total epochs trained: {total_epochs}')

# Save the loss, validation loss, accuracy, and validation accuracy to a CSV file
with open('training_metrics.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Epoch', 'Training Loss', 'Validation Loss', 'Training Accuracy', 'Validation Accuracy'])
    for epoch in range(total_epochs):
        training_loss = losses[epoch] if epoch < len(losses) else ''
        validation_loss = val_losses[epoch] if epoch < len(val_losses) else ''
        training_accuracy = accuracies[epoch] if epoch < len(accuracies) else ''
        validation_accuracy = val_accuracies[epoch] if epoch < len(val_accuracies) else ''
        csvwriter.writerow([epoch + 1, training_loss, validation_loss, training_accuracy, validation_accuracy])

# Plot the loss and validation loss over epochs
plt.figure(figsize=(10, 6))
plt.plot(range(total_epochs), losses, label='Training Loss')
plt.plot(range(total_epochs), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Decay Over Epochs')
plt.legend()
plt.show()

# Plot the accuracy and validation accuracy over epochs
plt.figure(figsize=(10, 6))
plt.plot(range(total_epochs), accuracies, label='Training Accuracy')
plt.plot(range(total_epochs), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Epochs')
plt.legend()
plt.show()


We can plot loss and validation accuracy during the training to check, for instance, for overfitting.

We save (and/or load) the final trained model.

In [ ]:
model.save(path + 'model.AM.h5')

In [ ]:
model = load_model(path + 'model.AM.h5')

Finally, we evaluate the training on the testing dataset, which is the 10th batch of data

In [ ]:
# Set the iteration number
i = 10

# Initialize the ImaFile object with specified folder, number of samples, and model name for the test simulations
file_sim = ImaFile(simulations_folder=path_sim + 'AM/Simulations' + str(i), nr_samples=198, model_name='Marth-3epoch-CEU')

# Read the simulation data for the test set
gene_sim_test = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=2000)

# Filter the simulation data to include only those with a frequency of 0.01
gene_sim_test.filter_freq(0.01)

# Sort the filtered simulation data by 'rows_freq'
gene_sim_test.sort('rows_freq')

# Resize the simulation data to a matrix with dimensions 198x192
gene_sim_test.resize((198, 192))

# Convert the simulation data, applying a flip transformation
gene_sim_test.convert(flip=True)

# Create a random subset of the test simulation data
rnd_idx = get_index_random(gene_sim_test)
gene_sim_test.subset(rnd_idx)

# Convert the targets of the test simulation data to binary
gene_sim_test.targets = to_binary(gene_sim_test.targets)


Let's report loss and accuracy on the testing set.

In [ ]:

net_LCT.test = model.evaluate(gene_sim_test.data, gene_sim_test.targets, batch_size=None, verbose=0)
print(net_LCT.test)  # it will report [loss, accuracy]



In [ ]:
# Add the following code for ROC and AUC calculation and plotting
y_pred_prob = model.predict(gene_sim_test.data)
fpr, tpr, _ = roc_curve(gene_sim_test.targets, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


# Convert probabilities to binary predictions
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(gene_sim_test.targets, y_pred).ravel()

# Calculate sensitivity (recall)
sensitivity = recall_score(gene_sim_test.targets, y_pred)

# Calculate specificity
specificity = tn / (tn + fp)

# Calculate precision
precision = precision_score(gene_sim_test.targets, y_pred)

# Calculate F1 score
f1 = f1_score(gene_sim_test.targets, y_pred)

# Print metrics
print(f"Sensitivity (Recall): {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1 Score: {f1:.2f}")

For a binary (or multiclass) classification, it is convenient to plot the confusion matrix after predicting the responses from the testing data.

In [ ]:
net_LCT.predict(gene_sim_test, model)
net_LCT.plot_cm(gene_sim_test.classes, text=True)